In [2]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime
import pandas as pd
import numpy as np
import random
import time
import os
import re


In [3]:
website = 'glassdoor'

now = datetime.now()

start_date = now.strftime('%m%d%Y')
# make data/playlist/ directory
os.makedirs(website +'/' + start_date , exist_ok=True)

chrome_path = r'C:/Users/user/scraper_sel/chromedriver.exe' # /chromedriver.exe
download_dir = r"C:/Users/user/projects/scrape_salary/data"


# read selenium service desc. this indicate the location of the chromedriver
service = Service(executable_path=chrome_path)  # "./chromedriver.exe")

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_dir}

chrome_options.add_experimental_option('prefs', prefs)


driver = webdriver.Chrome(service=service, chrome_options=chrome_options)
url = r'http://www.glassdoor.com'

driver.get(url)
driver.implicitly_wait(8)

keyword = 'Data Science'
location = 'Philippines'

# will be asked to log in

C:\Users\user\AppData\Local\Temp\ipykernel_2916\2005831545.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=service, chrome_options=chrome_options)


In [4]:

keyword_field = driver.find_element(By.ID, 'sc.keyword')
keyword_field.clear()
keyword_field.send_keys(keyword)

location_field = driver.find_element(By.ID, 'sc.location')

actions = ActionChains(driver)
actions.move_to_element(location_field)
actions.click(location_field)
actions.perform()

clear = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR,
    "div[aria-label='Clear']"
    )))

driver.execute_script("arguments[0].click();", clear)

location_field.clear()
location_field.send_keys(location)

find = driver.find_element(By.XPATH, "//button[@title='Submit search']")
find.click()

all_jobs = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH,
        "//strong[normalize-space()='See All Jobs']"
        )))

driver.execute_script("arguments[0].click();", all_jobs)

try:

    close_pop_up = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR,
        "span[alt='Close']"
        )))

    driver.execute_script("arguments[0].click();", close_pop_up)

    print('pop up closed')

except:
    print('no pop ups')

pop up closed


In [44]:

juju = None

if juju != None:
    pattern = r"\n\d\.\d"
    bella = re.sub(pattern, '', juju)
    bella
else:
    print('pass')

pass


In [67]:
page = 1

for _ in range(3):

        print(f'Harvesting page: {page}....', end="")

        mosaic = driver.find_element(By.XPATH, "//ul[@data-test='jlGrid']")\
                        .find_elements(By.TAG_NAME, "li")

        time.sleep(1 + 2*random.random())
        to_frame = []
        items = 0
        number = 0

        # create list container for clicking
        targets = driver.find_element(By.XPATH, "//ul[@data-test='jlGrid']")\
                        .find_elements(By.TAG_NAME, "li")
        
        for card in mosaic:

            time.sleep(1 + 2*random.random())

            job_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable(targets[number]))

            try:
                # JS click
                driver.execute_script('arguments[0].click();', job_button)
                items += 1
                summ_test = WebDriverWait(driver, 15).until(
                    EC.element_to_be_clickable((By.XPATH,
                    "//span[@class='css-2etp8b evpplnh1']//span[contains(text(),'Save')]"
                    )))

                try:
                    summary = driver.find_element(By.XPATH, "//div[@class='jobDescriptionContent desc']").text.strip()
                except:
                    summary = None
            except:
                print(f'click {number} failed')

            try:
                location = driver.find_element(By.CSS_SELECTOR, "div[data-test='emp-location']").text.strip()
                pattern = r"\n\d\.\d"
                location = re.sub(pattern, "", location)
            except:
                location = None

            try:
                title = driver.find_element(By.CSS_SELECTOR, "div[data-test='jobTitle']").text.strip()
            except:
                title = None
            
            try:
                company = driver.find_element(By.CSS_SELECTOR, "div[data-test='employerName']").text.strip()
                company = re.sub(pattern, "", company)
            except:
                company = None

            try:
                test_salary = card.find_elements(By.TAG_NAME, "span")[7].text.strip()
                if re.search('₱', test_salary):
                    salary = test_salary
                else:
                    salary = None
            except:
                salary = None

            try:
                job_url = card.find_element(By.CSS_SELECTOR,'a[data-test="job-link"]').get_attribute('href')
            except:
                job_url = None
            
            job_info = {
                'location': location,
                'title': title,
                'company': company,
                'salary': salary,
                'summary': summary,
                'job_url': job_url}

            to_frame.append(job_info)
            number += 1

        framed_df = pd.DataFrame(to_frame)
        framed_df.to_csv(website + '/' + start_date + '/page' + str(page) + '.csv', index=False)
        print('Saved!')

        try:
            next_page = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH,
                    "//button[@aria-label='Next']"
                    )))

            driver.execute_script("arguments[0].click();", next_page)        
            page += 1

        except:
            print(f"Exception hit, scaping ended at page {page - 1}")
            break

print('Harvest completed')  

Harvesting page: 1....Saved!
Harvesting page: 2....Saved!
Harvesting page: 3....Saved!
Harvest completed


In [71]:
read_df = pd.read_csv('glassdoor/10312022/page1.csv')
read_df

,location,title,company,salary,summary,job_url
0,NaN,Data Analyst,MoxiWorks0,₱35K - ₱40K (Employer est.),The Company\nMoxiWorks is a comprehensive open...,https://www.glassdoor.com/partner/jobListing.h...
1,NaN,Jr. Data Analyst,CloudLogic,NaN,Key tasks and responsibilities:\n• Managing ma...,https://www.glassdoor.com/partner/jobListing.h...
2,NaN,ESG Data Analyst,RIMES Technologies Corporation,NaN,Summary\nPosted:\n2022-09-27T11:37:51-04:00\nJ...,https://www.glassdoor.com/partner/jobListing.h...
3,NaN,Data Analyst (Junior Production Planning Officer),Bounty Fresh Food Inc.,₱17K (Employer est.),"√ Graduate of BS Information Technology, BS Co...",https://www.glassdoor.com/partner/jobListing.h...
4,NaN,"Data Analyst (Python, R, SQL)",Aspiree Inc.,₱60K - ₱90K (Employer est.),QUALIFICATIONS:\nBachelor’s Degree Graduate in...,https://www.glassdoor.com/partner/jobListing.h...
5,NaN,DATA ANALYST,GlobalQuest,NaN,Perform the following:\no Test Planning (Test ...,https://www.glassdoor.com/partner/jobListing.h...
6,NaN,Data Encoder Research project-based,De La Salle Medical and Health Sciences Institute,NaN,Specification:\nBachelor’s Degree\nData manage...,https://www.glassdoor.com/partner/jobListing.h...
7,NaN,Drug Safety Specialist,Parexel,NaN,Job Description:\nResponsible for capturing an...,https://www.glassdoor.com/partner/jobListing.h...
8,NaN,"Senior Manager, Data Science",AIA,NaN,At AIA we’ve started an exciting movement to c...,https://www.glassdoor.com/partner/jobListing.h...
9,NaN,Sr. Data Analyst,"GoodTech Holdings, Inc.0",NaN,GoodTech is on a mission to serve the unbanked...,https://www.glassdoor.com/partner/jobListing.h...


In [54]:
framed_df

,location,title,company,salary,summary,job_url
0,Lovemyjob,Data Analyst,Lovemyjob,None,"Full Time, With Full Benefits and Profit Shari...",https://www.glassdoor.com/partner/jobListing.h...
1,Uber,Data Analytics and Visualization Specialist,Uber,None,Uber Centers of Excellence (COEs) are contact ...,https://www.glassdoor.com/partner/jobListing.h...
2,PSM Ltd.0,Data Analyst Associate,PSM Ltd.0,₱55K - ₱85K (Employer est.),Role: Permanent\nDepartment: Information Techn...,https://www.glassdoor.com/partner/jobListing.h...
3,In1go Technologies,Data Analyst (BS Mathematics),In1go Technologies,None,Job description\nResponsibilities:\nLiaising w...,https://www.glassdoor.com/partner/jobListing.h...
4,Esco Lifesciences Group,Data Scientist - AI Developer,Esco Lifesciences Group,None,Job Description\nEsco is currently looking for...,https://www.glassdoor.com/partner/jobListing.h...
5,"Afni, Inc.",Data Scientist,"Afni, Inc.",None,Position Purpose:\nThe Data Scientist is Afni'...,https://www.glassdoor.com/partner/jobListing.h...
6,ALL ABOUT PEOPLE CONSULTING0,DATA ANALYST,ALL ABOUT PEOPLE CONSULTING0,₱60K (Employer est.),Job Qualifications:\nGraduate of Bachelor's De...,https://www.glassdoor.com/partner/jobListing.h...
7,Emapta,Data Analyst,Emapta,None,Job Information\nIndustry\nEmployment - Recrui...,https://www.glassdoor.com/partner/jobListing.h...
8,Meralco,Data Scientist,Meralco,None,College graduate with a degree in Computer Sci...,https://www.glassdoor.com/partner/jobListing.h...
9,Bayer,Data Scientist,Bayer,None,Intelligent Automation - PM Sr. Expert (Data S...,https://www.glassdoor.com/partner/jobListing.h...
